# New York Movies Scene


In [1]:
import pandas as pd
import numpy as np
import re
from pandas import Series, DataFrame
movie_scene = pd.read_csv('Film_Permits.csv',skipinitialspace = True)
movie_scene = pd.DataFrame(movie_scene)

In [2]:
movie_scene.dropna(inplace=True)

In [3]:
clean_movie_scene = movie_scene
clean_movie_scene['CommunityBoard(s)'] = clean_movie_scene['CommunityBoard(s)'].str.replace(' ','')
clean_movie_scene['PolicePrecinct(s)'] = clean_movie_scene['PolicePrecinct(s)'].str.replace(' ','')

In [4]:
def setMean(columns):
    mean = []
    for cell in clean_movie_scene[columns]:
    #     cell to string
        getCell = str(cell)
        #   if the first character is not a number remove it
        if getCell[0].isnumeric() == False:
            getCell = getCell[1:]
    #     cell to numpy array seperated by comma
        seperateCell = np.array(np.fromstring(getCell, dtype=int, sep=','))
    #     appaned mean
        mean.append(int(np.mean(seperateCell)))
    return mean

clean_movie_scene['CommunityBoard(s)'] =  setMean('CommunityBoard(s)')
clean_movie_scene['PolicePrecinct(s)'] = setMean('PolicePrecinct(s)')
clean_movie_scene

,EventID,EventType,StartDateTime,EndDateTime,EnteredOn,EventAgency,ParkingHeld,Borough,CommunityBoard(s),PolicePrecinct(s),Category,SubCategoryName,Country,ZipCode(s)
0,446040,Shooting Permit,10/19/2018 02:00:00 PM,10/20/2018 04:00:00 AM,10/16/2018 11:57:27 AM,"Mayor's Office of Film, Theatre & Broadcasting",THOMPSON STREET between PRINCE STREET and SPRI...,Manhattan,2,1,Television,Cable-episodic,United States of America,10012
1,446168,Shooting Permit,10/19/2018 02:00:00 PM,10/20/2018 02:00:00 AM,10/16/2018 07:03:56 PM,"Mayor's Office of Film, Theatre & Broadcasting",MARBLE HILL AVENUE between WEST 227 STREET an...,Manhattan,10,42,Film,Feature,United States of America,"10034, 10463"
2,186438,Shooting Permit,10/30/2014 07:00:00 AM,10/31/2014 02:00:00 AM,10/27/2014 12:14:15 PM,"Mayor's Office of Film, Theatre & Broadcasting",LAUREL HILL BLVD between REVIEW AVENUE and RUS...,Queens,3,106,Television,Episodic series,United States of America,11378
3,445255,Shooting Permit,10/20/2018 07:00:00 AM,10/20/2018 06:00:00 PM,10/09/2018 09:34:58 PM,"Mayor's Office of Film, Theatre & Broadcasting",JORALEMON STREET between BOERUM PLACE and COUR...,Brooklyn,2,84,Still Photography,Not Applicable,United States of America,11201
4,128794,Theater Load in and Load Outs,11/16/2013 12:01:00 AM,11/17/2013 06:00:00 AM,11/07/2013 03:48:28 PM,"Mayor's Office of Film, Theatre & Broadcasting",WEST 31 STREET between 7 AVENUE and 8 AVENUE...,Manhattan,4,14,Theater,Theater,United States of America,"10001, 10121"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74443,605415,Shooting Permit,10/09/2021 06:00:00 AM,10/09/2021 03:00:00 PM,10/03/2021 04:54:51 PM,"Mayor's Office of Film, Theatre & Broadcasting",BEAVER STREET between BROADWAY and SOUTH WILLI...,Manhattan,2,27,WEB,Not Applicable,United States of America,"10002, 10004, 10005, 10006, 10013, 10019, 1002..."
74444,549343,Shooting Permit,10/26/2020 07:00:00 AM,10/26/2020 07:00:00 PM,10/21/2020 09:20:36 AM,"Mayor's Office of Film, Theatre & Broadcasting",MESEROLE AVENUE between DIAMOND STREET and JEW...,Brooklyn,1,94,Television,Episodic series,United States of America,11222
74445,532104,Shooting Permit,02/11/2020 07:00:00 AM,02/11/2020 10:00:00 PM,02/10/2020 11:58:59 AM,"Mayor's Office of Film, Theatre & Broadcasting",EAST 51 STREET between MADISON AVENUE and PA...,Manhattan,4,43,Television,Episodic series,United States of America,"10017, 10022, 10171, 11222"
74446,575935,Shooting Permit,05/12/2021 09:00:00 AM,05/12/2021 11:00:00 PM,05/06/2021 12:07:03 PM,"Mayor's Office of Film, Theatre & Broadcasting",WEST 47 STREET between 8 AVENUE and 7 AVENUE...,Manhattan,4,16,Film,Feature,United States of America,"10018, 10019, 10020, 10036"
